In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

Using TensorFlow backend.


In [4]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

NameError: name 'np' is not defined

In [3]:
# load dataset
from google.colab import files
uploaded = files.upload()

Saving sms.csv to sms.csv


In [0]:
import pandas as pd
import io
df = pd.read_csv(io.StringIO(uploaded['sms.csv'].decode("ISO-8859-1")))

In [3]:
df.head()

NameError: name 'df' is not defined

In [6]:


df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
v1    5572 non-null object
v2    5572 non-null object
dtypes: object(2)
memory usage: 87.1+ KB


In [2]:


sns.countplot(df.v1)
plt.xlabel('Label')
plt.title('Number of ham and spam messages')



NameError: name 'sns' is not defined

In [1]:
X = df.v2
Y = df.v1
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

NameError: name 'df' is not defined

In [0]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3)

In [0]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [0]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [12]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

W0817 06:51:42.253232 140339071354752 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0817 06:51:42.301572 140339071354752 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0817 06:51:42.303235 140339071354752 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0817 06:51:42.580472 140339071354752 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0817 06:51:42.591722 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 150)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 150, 50)           50000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257       
__________

In [13]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Train on 3120 samples, validate on 780 samples
Epoch 1/10
3120/3120 [==============================] - 7s 2ms/step - loss: 0.3652 - acc: 0.8590 - val_loss: 0.2069 - val_acc: 0.8897
Epoch 2/10
3120/3120 [==============================] - 6s 2ms/step - loss: 0.1299 - acc: 0.9606 - val_loss: 0.0837 - val_acc: 0.9782
Epoch 3/10
3120/3120 [==============================] - 6s 2ms/step - loss: 0.0518 - acc: 0.9853 - val_loss: 0.0581 - val_acc: 0.9872
Epoch 4/10
3120/3120 [==============================] - 6s 2ms/step - loss: 0.0339 - acc: 0.9891 - val_loss: 0.0566 - val_acc: 0.9872
Epoch 5/10
3120/3120 [==============================] - 6s 2ms/step - loss: 0.0236 - acc: 0.9923 - val_loss: 0.0600 - val_acc: 0.9885


In [0]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [15]:
accr = model.evaluate(test_sequences_matrix,Y_test)

1672/1672 [==============================] - 1s 653us/step


In [16]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.067
  Accuracy: 0.984
